# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [19]:
import networkx
import urllib2
homer = urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

In [11]:
def read_nodes(gfile):
    nodes=[]
    next=gfile.readline()
    while next != "\n": #Runs till the end of Characters descritption
        if len(next)>3: #Selecting only those lines with greater 3 length
            if next[2]==" ": #The third element is always space in these lines
                nodes.append(next[0:2])
        next=gfile.readline()
    return nodes
    pass

Next implement a function to read in the edges from the input file.

In [12]:
def read_edges(gfile):
    edges=[]
    next=gfile.readline() #Continues from the \n line where the nodes function stopped
    while next != "":
        if ":" in next: #all the required lines contain ':'
            next=next.split(":")[1]
            next=next.strip()
            next=next.split(';')
            for i in range(0,len(next)): #making pairs of connected characters
                edge_set=next[i]
                edge_set=edge_set.split(',')
                for j in range(0,len(edge_set)):
                    for k in range(j+1,len(edge_set)):
                        edge=(edge_set[j],edge_set[k])
                        edges.append(edge)
        next=gfile.readline()
    return edges
    pass

The following code should now correctly create the graph.

In [13]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [14]:
def Search(graph,root):
    dfs=[root]   #List of current state of the dfs tree
    visited=[root] #list of all the visited nodes
    while dfs: #while dfs is not empty
        neigh=graph.neighbors(dfs[len(dfs)-1])
        neigh.sort()
        neigh_found=0
        for i in neigh: #chooses the neighbor of current node which is not visited
            if i not in visited:
                visited.append(i)
                dfs.append(i)
                neigh_found=1
                break
        if neigh_found==0: #if all neighbors are visited, pop out the node from dfs
            dfs.pop()
    return visited[1:]

We will check the correctness of your code by verifying that it correctly computes the DFS tree starting at Ulysses (node `OD`).

In [15]:
ulysses = Search(G, 'OD')

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [16]:
def connected_components(graph):
    nodes=graph.nodes()
    nodes.sort()
    connected=[]
    i=0 #Tells the number of connected groups found
    while nodes:
        conn=Search(graph,nodes[0]) #first element is always the un-visite
        connected.extend([[nodes[0]]+conn])
        nodes_new = [x for x in nodes if x not in connected[i]] #Removing all the visited nodes from nodes[]
        nodes=nodes_new
        i=i+1 
    return(connected)
    pass

We will check correctness of your code by verifying that your output list is identical to our solution.

In [17]:
character_interactions = connected_components(G)

As a preliminary check, you should find that the following statements are all true.

In [18]:
component_sizes = [len(c) for c in character_interactions]
print "There are 12 connected components in the Iliad:", len(component_sizes) == 12
print "The giant component has size 542:", max(component_sizes) == 542
print "There are 5 isolated characters:", len([c for c in component_sizes if c == 1]) == 5

There are 12 connected components in the Iliad: True
The giant component has size 542: True
There are 5 isolated characters: True
